In [93]:
import numpy as np
import time
from memory_profiler import memory_usage

import plotting_standards.colors as colors
import plotting_standards.utils as plotting_utils

from plotly.subplots import make_subplots
import plotly.offline as plotly
import plotly.graph_objs as go

%matplotlib inline
plotly.init_notebook_mode(connected=True)

In [3]:
def euler_ode_solver(f, start_x, stop_x, start_y, n_steps):
    step_size = (stop_x - start_x) / n_steps
    sampled_points = [(start_x, start_y)]
    
    for step in range(1, n_steps + 1):
        last_x, last_y = sampled_points[-1]
        sampled_points.append((last_x + step_size, last_y + step_size * f(last_y, last_x)))
        
    return sampled_points

In [4]:
w = np.array([3.0])
b = 1.0

def f(y, x):
    return np.matmul(w, y) + b

def y(t):
    return np.exp(w * t + np.log(b / w)) - b / w

# Empirically Showing Theoretical Convergence

In [5]:
errors = []
for n in range(10, 1000, 10):
    experimental = np.array(euler_ode_solver(f, 0.0, 1.0, np.array([0.0]), n))[:,1] 
    gt = y(np.arange(0, n + 1) / n)
    errors.append(np.abs(experimental - gt)[-1][0])

In [6]:
n_runs = 10
t_costs = []
for n in range(10, 1000, 10):
    t_cost_avg = 0.0
    for _ in range(n_runs):
        start = time.time()
        np.array(euler_ode_solver(f, 0.0, 1.0, np.array([0.0]), n))
        t_cost_avg += time.time() - start
    t_costs.append(t_cost_avg / n_runs)

In [10]:
n_runs = 10
m_costs = []
for n in range(10, 1000, 10):
    m_cost_avg = 0.0
    for _ in range(n_runs):
        mem_usage = memory_usage((euler_ode_solver, (f, 0.0, 1.0, np.array([0.0]), n)))
        m_cost_avg += max(mem_usage)
    m_costs.append(m_cost_avg / n_runs)

In [192]:
plot_1 = go.Scatter(
    x=np.arange(10, 1000), y=t_costs, line={"smoothing": 0.5, "shape": "spline", 'color': colors.core_1})
plot_2 = go.Scatter(
    x=np.arange(10, 1000), y=m_costs, line={"smoothing": 0.5, "shape": "spline", 'color': colors.core_2})
plot_3 = go.Scatter(
    x=np.arange(10, 1000), y=errors, line={"smoothing": 0.5, "shape": "spline", 'color': colors.core_3})

fig = make_subplots(rows=1, cols=3, subplot_titles=("Steps vs. Time", "Steps vs. Memory", "Steps vs. Error"))
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.add_trace(plot_3, row=1, col=3)

fig.update_layout(showlegend=False, height=300)
plotting_utils.set_layout(fig['layout'])
fig['layout']['xaxis']['title']['text'] = "Steps"
fig['layout']['yaxis']['title']['text'] = "Time"
fig['layout']['xaxis2']['title']['text'] = "Steps"
fig['layout']['yaxis2']['title']['text'] = "Memory"
fig['layout']['xaxis3']['title']['text'] = "Steps"
fig['layout']['yaxis3']['title']['text'] = "Absolute Error"

fig.show()

In [193]:
with open("/home/fkwang/Downloads/ode_solver_baseline_independent_step.svg", 'wb') as f:
    f.write(fig.to_image(format="svg"))

In [200]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("Error vs. Time", "Error vs. Memory"))

plot_1 = go.Scatter(
    x=errors, y=t_costs, line={"smoothing": 0.5, "shape": "spline", 'color': colors.core_1})
plot_2 = go.Scatter(
    x=errors, y=m_costs, line={"smoothing": 0.5, "shape": "spline", 'color': colors.core_2})

fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)

fig.update_layout(showlegend=False, height=300)
plotting_utils.set_layout(fig['layout'])
fig['layout']['xaxis']['title']['text'] = "Absolute Error"
fig['layout']['yaxis']['title']['text'] = "Time"
fig['layout']['xaxis2']['title']['text'] = "Absolute Error"
fig['layout']['yaxis2']['title']['text'] = "Memory"

fig.show()

In [202]:
with open("/home/fkwang/Downloads/ode_solver_baseline_independent_error.svg", 'wb') as f:
    f.write(fig.to_image(format="svg"))